In [5]:
import numpy as np 
import pandas as pd
from pandas import  DataFrame,Series
from sklearn.preprocessing import LabelBinarizer,StandardScaler
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

#导入所有相关数据
train = pd.read_csv('C:/Users/Administrator.-20170614IXISUQ/Desktop/幸福感/happiness_train_complete.csv', encoding='gbk')
test_data = pd.read_csv('C:/Users/Administrator.-20170614IXISUQ/Desktop/幸福感/happiness_test_complete.csv', encoding='gbk')
submit = pd.read_csv('C:/Users/Administrator.-20170614IXISUQ/Desktop/幸福感/happiness_submit.csv', encoding='gbk')


In [6]:
#排除异常值-8
train = train[train['happiness']!=-8]

#处理时间序列获取被调查人群年龄
from datetime import datetime
train['age'] = pd.to_datetime(train['survey_time']).dt.year - train['birth']
test_data['age'] = pd.to_datetime(test_data['survey_time']).dt.year - test_data['birth']

#删除缺失值较多且感觉无具体含义的数据

ind1 = ['id','edu_other','edu_status','edu_yr','join_party','property_other','work_status','work_yr','work_type','work_manage',
       'invest_other','survey_time','birth']
X = train.drop(ind1,axis=1)
X_test_data = test_data.drop(ind1,axis=1)
#X.info(verbose=True,null_counts=True)

#对所有缺失行进行补充充填
#因样本中有着多种与字段解释不匹配的负数存在， 统一将其负数改为-8
X['hukou_loc'] = X['hukou_loc'].fillna(4)
X = X.fillna(-8)
X[X<0]=-8
X.head()
X_test_data['hukou_loc'] = X_test_data['hukou_loc'].fillna(4)
X_test_data = X_test_data.fillna(-8)
X_test_data[X_test_data<0]=-8


In [7]:
#分离标签转换标签
X_train1 = X.drop(['happiness'],axis=1)
X_label = X['happiness']

In [8]:

#因各特征差距影响较大，进行标准化
std = StandardScaler().fit(X_train1.values)
X_train_std = std.transform(X_train1)
std_1 = StandardScaler().fit(X_test_data.values)
X_test_std = std_1.transform(X_test_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [10]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
lgb = LGBMRegressor(random_state=0)
lgb_param_grid = {'n_estimators':[300,500,800],'learning_rate':[0.01,0.1]}
lgb_grid = GridSearchCV(lgb,lgb_param_grid,n_jobs=25,cv=10,verbose=1)
xgb = XGBRegressor(random_state=0)
xgb_param_grid = {'n_estimators':[300,500,800],'learning_rate':[0.01,0.1]}
xgb_grid = GridSearchCV(xgb,xgb_param_grid,n_jobs=25,cv=10,verbose=1)
gb = GradientBoostingRegressor(random_state=0)
gb_param_grid = {'n_estimators':[300,500,800],'learning_rate':[0.01,0.1]}
gb_grid = GridSearchCV(gb,gb_param_grid,n_jobs=25,cv=10,verbose=1)

In [11]:
lgb_grid.fit(X_train_std,X_label)
xgb_grid.fit(X_train_std,X_label)
gb_grid.fit(X_train_std,X_label)
lgb_r = lgb_grid.best_estimator_
xgb_r = xgb_grid.best_estimator_
gb_r = gb_grid.best_estimator_

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  60 out of  60 | elapsed:  1.2min finished


Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  60 out of  60 | elapsed:  4.1min finished
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[09:56:40] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  60 out of  60 | elapsed: 10.1min finished


In [18]:
lgb_score = cross_val_score(lgb_r,X_train_std,X_label,cv=5,scoring='neg_mean_squared_error')
xgb_score = cross_val_score(xgb_r,X_train_std,X_label,cv=5,scoring='neg_mean_squared_error')
gb_score = cross_val_score(gb_r,X_train_std,X_label,cv=5,scoring='neg_mean_squared_error')


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:27:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:27:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:27:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:28:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:28:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [16]:
print(lgb_score.mean())
print(xgb_score.mean())
print(gb_score.mean())

-0.4654849561399127
-0.4648760786646041
-0.4670444668565511


In [19]:
y_model =xgb_r.predict(X_test_std)

In [21]:
submit['happiness']=y_model
submit.to_csv('gb_r.csv', index=None)